In [147]:
#import dependencies 
import pandas as pd 
import requests 
import json
import datetime
import time 

In [148]:
#Find today's date
today = datetime.datetime.now()
#find the start and end of today in milliseconds
start_today = int(today.replace(hour=0, minute=0, second=0, microsecond=0).timestamp()*1000)
end_today = int(today.timestamp()*1000)

In [156]:
#Calculate the Start and End of the day for the previous 7 days
#Infer the days of the week from the calculated dates
#Puts all data in pandas dataframe 'lastweek'
lastweeks_dates = [f'{today.isoformat()}']
dt_dates = [today]
start_times = [start_today]
end_times = [end_today]
thisdaystr=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_num =[today.weekday()]
week_day = [thisdaystr[today.weekday()]]

for n in range(1,8):
    #get previous day(s) date isoformat
    delta = datetime.timedelta(days=n)
    previous_day = today - delta
    date = previous_day.isoformat()
    #get previous day(s) start time (milliseconds)
    start_previous = int(previous_day.replace(hour=0, minute=0, second=0, microsecond=0).timestamp())*1000
    #get previous day(s) end time (milliseconds)
    end_previous = int(previous_day.replace(hour=23,minute=59, second=59, microsecond=999).timestamp())*1000
    #get the day of the week 
    day_num = previous_day.weekday()
    day_str = thisdaystr[day_num]
    #append all calculated data
    lastweeks_dates.append(date)
    dt_dates.append(previous_day)
    start_times.append(start_previous)
    end_times.append(end_previous)
    week_day.append(day_str)
    weekday_num.append(day_num)
    
last_wks_dates = {
    "iso": lastweeks_dates,
    "datetime": dt_dates,
    "start": start_times,
    "end": end_times,
    "dayofweek": week_day,
    "daynumber": weekday_num
}

lastweek = pd.DataFrame(last_wks_dates)


In [157]:
lastweek

,iso,datetime,start,end,dayofweek,daynumber
0,2021-05-10T18:47:35.248581,2021-05-10 18:47:35.248581,1620630000000,1620697655248,Monday,0
1,2021-05-09T18:47:35.248581,2021-05-09 18:47:35.248581,1620543600000,1620629999000,Sunday,6
2,2021-05-08T18:47:35.248581,2021-05-08 18:47:35.248581,1620457200000,1620543599000,Saturday,5
3,2021-05-07T18:47:35.248581,2021-05-07 18:47:35.248581,1620370800000,1620457199000,Friday,4
4,2021-05-06T18:47:35.248581,2021-05-06 18:47:35.248581,1620284400000,1620370799000,Thursday,3
5,2021-05-05T18:47:35.248581,2021-05-05 18:47:35.248581,1620198000000,1620284399000,Wednesday,2
6,2021-05-04T18:47:35.248581,2021-05-04 18:47:35.248581,1620111600000,1620197999000,Tuesday,1
7,2021-05-03T18:47:35.248581,2021-05-03 18:47:35.248581,1620025200000,1620111599000,Monday,0


In [151]:
#build a query for today's data 
UID = "me"
OAUTH_TOKEN="ya29.a0AfH6SMDcIQvu9DkucaB0NCLXWv63Gjxrr1_TMK58BAroNbyhFayP3pFAWuzxr8zJ40pwUEBSMksFmF2MwOAPqhqjEyWb6vKg4JPf-iMjOLkQ46r89p-P5jw07Zg_jBZS6MTeK01G4XHEG86f1ctnI5VeEruIGA"
url = f'https://www.googleapis.com/fitness/v1/users/me/dataset:aggregate'

headers = { 'content-type': 'application/json',
            'Authorization': 'Bearer %s' % OAUTH_TOKEN}
total_steps=[]
for i in range(0,8):
    request_body={
        "aggregateBy": [{
            "dataSourceId":"derived:com.google.step_count.delta:com.google.android.gms:estimated_steps"}],
        "bucketByTime": { "durationMillis": 86400000 },
        "startTimeMillis": str(lastweek['start'][i]),
        "endTimeMillis": str(lastweek['end'][i])}
    r = requests.post(url, headers=headers, json=request_body)
    response = r.json()
    #extract daily number of the steps
    steps = response['bucket'][0]['dataset'][0]['point'][0]['value'][0]['intVal']
    total_steps.append(steps)
    
print(total_steps)

[11241, 4733, 10917, 5423, 10819, 10041, 17060, 16894]


In [158]:
lastweek

,iso,datetime,start,end,dayofweek,daynumber
0,2021-05-10T18:47:35.248581,2021-05-10 18:47:35.248581,1620630000000,1620697655248,Monday,0
1,2021-05-09T18:47:35.248581,2021-05-09 18:47:35.248581,1620543600000,1620629999000,Sunday,6
2,2021-05-08T18:47:35.248581,2021-05-08 18:47:35.248581,1620457200000,1620543599000,Saturday,5
3,2021-05-07T18:47:35.248581,2021-05-07 18:47:35.248581,1620370800000,1620457199000,Friday,4
4,2021-05-06T18:47:35.248581,2021-05-06 18:47:35.248581,1620284400000,1620370799000,Thursday,3
5,2021-05-05T18:47:35.248581,2021-05-05 18:47:35.248581,1620198000000,1620284399000,Wednesday,2
6,2021-05-04T18:47:35.248581,2021-05-04 18:47:35.248581,1620111600000,1620197999000,Tuesday,1
7,2021-05-03T18:47:35.248581,2021-05-03 18:47:35.248581,1620025200000,1620111599000,Monday,0


In [155]:
lastweek

In [101]:
response = r.json()

In [122]:
steps = response['bucket'][0]['dataset'][0]['point'][0]['value'][0]['intVal']

11218

In [ ]:
#get the current week's date and time range 
today = datetime.date.today()
weekday = today.weekday()
start_delta = datetime.timedelta(days=weekday)
start_of_week = today - start_delta
week_dates = []
for day in range(7):
    week_dates.append(start_of_week + datetime.timedelta(days=day))
    
week_dates
[datetime.date(2013, 10, 28), datetime.date(2013, 10, 29), datetime.date(2013, 10, 30),     datetime.date(2013, 10, 31), datetime.date(2013, 11, 1), datetime.date(2013, 11, 2), datetime.date(2013, 11, 3)]
>>> week_dates[0], week_dates[-1]
(datetime.date(2013, 10, 28), datetime.date(2013, 11, 3)) <--- Monday, Sunday

In [2]:
{
"dataStreamId": "exampleDataSourceId",
...
"dataType": {
  "name": "com.google.step_count.delta"
}

SyntaxError: invalid syntax (<ipython-input-2-9092b85e3037>, line 4)

In [62]:
UID = "me"
OAUTH_TOKEN="ya29.a0AfH6SMDAECMGpLmvKHBq-cIBAbmjY_xL8Z9eJiYtJvLt45Fe_f2BbZiIieE5lmhAsdttmnT6IfsiotT3O4UlIz9sQEDpyVPN6HyeClB_BNXaJ9vrX5UWCvkmohCEXFc8mjW6ZCU_fHFNKRYvB88zk_5ZmVnvJQ"
url = f'https://www.googleapis.com/fitness/v1/users/me/sessions'

headers = { 'content-type': 'application/json',
            'Authorization': 'Bearer %s' % OAUTH_TOKEN}
request_body={"startTime":"2021-05-01T00:00:00.000Z",
              "endTime":"2021-05-30T00:00:00.000Z"
}

r = requests.get(url, headers=headers, params=request_body)

print(r.status_code)
print(r.content)

200
b'{\n  "session": [\n    {\n      "id": "0c07232b8ac4d889:watch-activemode:walking:1619826808774",\n      "name": "Afternoon walk",\n      "description": "",\n      "startTimeMillis": "1619826808774",\n      "endTimeMillis": "1619834454289",\n      "modifiedTimeMillis": "1619835997396",\n      "application": {\n        "packageName": "com.google.android.apps.fitness"\n      },\n      "activityType": 7\n    },\n    {\n      "id": "0c07232b8ac4d889:watch-activemode:running.treadmill:1619910321870",\n      "name": "Afternoon treadmill run",\n      "description": "",\n      "startTimeMillis": "1619910321870",\n      "endTimeMillis": "1619911861538",\n      "modifiedTimeMillis": "1619915135560",\n      "application": {\n        "packageName": "com.google.android.apps.fitness"\n      },\n      "activityType": 58\n    },\n    {\n      "id": "3b9e44d711382023:watch-activemode:walking.treadmill:1620068894525",\n      "name": "Lunch treadmill walk",\n      "description": "",\n      "startTim

In [63]:
response = r.json()

In [64]:
response.keys()

dict_keys(['session', 'deletedSession'])

In [65]:
response['session']

[{'id': '0c07232b8ac4d889:watch-activemode:walking:1619826808774',
  'name': 'Afternoon walk',
  'description': '',
  'startTimeMillis': '1619826808774',
  'endTimeMillis': '1619834454289',
  'modifiedTimeMillis': '1619835997396',
  'application': {'packageName': 'com.google.android.apps.fitness'},
  'activityType': 7},
 {'id': '0c07232b8ac4d889:watch-activemode:running.treadmill:1619910321870',
  'name': 'Afternoon treadmill run',
  'description': '',
  'startTimeMillis': '1619910321870',
  'endTimeMillis': '1619911861538',
  'modifiedTimeMillis': '1619915135560',
  'application': {'packageName': 'com.google.android.apps.fitness'},
  'activityType': 58},
 {'id': '3b9e44d711382023:watch-activemode:walking.treadmill:1620068894525',
  'name': 'Lunch treadmill walk',
  'description': '',
  'startTimeMillis': '1620068894525',
  'endTimeMillis': '1620070320832',
  'modifiedTimeMillis': '1620073438177',
  'application': {'packageName': 'com.google.android.apps.fitness'},
  'activityType': 95

In [69]:
#List sessions from a certain date range 
UID = "me"
OAUTH_TOKEN="ya29.a0AfH6SMDaKLr0cqu-AMoL9C7YzsBG89QaTNhC5zVO6sKtjv2BUCfMR920BmYfFHis0crxJj8xWLLbe3KjIRO5WdpVPLpi5eoXx3Wvu3G9H1b6BTgZ6ErJ6vp9Iqyfku2DP_BkTEJOvmNrzWxYOXi4oVSHX_N2Qw"
url = f'https://www.googleapis.com/fitness/v1/users/me/sessions'

headers = { 'content-type': 'application/json',
            'Authorization': 'Bearer %s' % OAUTH_TOKEN}

request_body={
    "startTime":"2021-05-02T00:00:00.000Z",
    "endTime":"2021-05-08T23:59:59.999Z"
}

r = requests.get(url, headers=headers)

print(r.status_code)
print(r.content)

200
b'{\n  "session": [\n    {\n      "id": "9079c8f1c15cd125:activemode:walking.fitness:1618241849733",\n      "name": "Morning fitness walk",\n      "description": "",\n      "startTimeMillis": "1618241849757",\n      "endTimeMillis": "1618245502437",\n      "modifiedTimeMillis": "1618256076177",\n      "application": {\n        "packageName": "com.google.android.apps.fitness"\n      },\n      "activityType": 93\n    },\n    {\n      "id": "0c07232b8ac4d889:watch-activemode:running.treadmill:1619718703608",\n      "name": "Morning treadmill run",\n      "description": "",\n      "startTimeMillis": "1619718703608",\n      "endTimeMillis": "1619720707208",\n      "modifiedTimeMillis": "1619754049928",\n      "application": {\n        "packageName": "com.google.android.apps.fitness"\n      },\n      "activityType": 58\n    },\n    {\n      "id": "13f43919-7cc0-492a-ab10-f3c021e7195e",\n      "name": "walk stretch",\n      "description": "",\n      "startTimeMillis": "1620234000000",\n  

In [70]:
last_week = r.json() 
print(last_week.keys())

dict_keys(['session', 'deletedSession', 'nextPageToken'])


In [71]:
last_week

{'session': [{'id': '9079c8f1c15cd125:activemode:walking.fitness:1618241849733',
   'name': 'Morning fitness walk',
   'description': '',
   'startTimeMillis': '1618241849757',
   'endTimeMillis': '1618245502437',
   'modifiedTimeMillis': '1618256076177',
   'application': {'packageName': 'com.google.android.apps.fitness'},
   'activityType': 93},
  {'id': '0c07232b8ac4d889:watch-activemode:running.treadmill:1619718703608',
   'name': 'Morning treadmill run',
   'description': '',
   'startTimeMillis': '1619718703608',
   'endTimeMillis': '1619720707208',
   'modifiedTimeMillis': '1619754049928',
   'application': {'packageName': 'com.google.android.apps.fitness'},
   'activityType': 58},
  {'id': '13f43919-7cc0-492a-ab10-f3c021e7195e',
   'name': 'walk stretch',
   'description': '',
   'startTimeMillis': '1620234000000',
   'endTimeMillis': '1620237600000',
   'modifiedTimeMillis': '1620240516693',
   'application': {'packageName': 'com.google.android.apps.fitness'},
   'activityType

In [75]:
UID = "me"
OAUTH_TOKEN="ya29.a0AfH6SMDaKLr0cqu-AMoL9C7YzsBG89QaTNhC5zVO6sKtjv2BUCfMR920BmYfFHis0crxJj8xWLLbe3KjIRO5WdpVPLpi5eoXx3Wvu3G9H1b6BTgZ6ErJ6vp9Iqyfku2DP_BkTEJOvmNrzWxYOXi4oVSHX_N2Qw"
url = f'https://www.googleapis.com/fitness/v1/users/me/dataset:aggregate'

headers = { 'content-type': 'application/json',
            'Authorization': 'Bearer %s' % OAUTH_TOKEN}

request_body={
  "aggregateBy": [{
    "dataSourceId":
      "derived:com.google.step_count.delta:com.google.android.gms:estimated_steps"
  }],
  "bucketByTime": { "durationMillis": 86400000 },
  "startTimeMillis": '1618241849757',
  "endTimeMillis": '1618245502437'
}


r = requests.post(url, headers=headers, json=request_body)

print(r.status_code)
print(r.content)

200
b'{\n  "bucket": [\n    {\n      "startTimeMillis": "1618241849757",\n      "endTimeMillis": "1618245502437",\n      "dataset": [\n        {\n          "dataSourceId": "derived:com.google.step_count.delta:com.google.android.gms:aggregated",\n          "point": [\n            {\n              "startTimeNanos": "1618241849757000000",\n              "endTimeNanos": "1618245502437000000",\n              "dataTypeName": "com.google.step_count.delta",\n              "originDataSourceId": "raw:com.google.step_count.cumulative:OnePlus:GM1915:33e928a4fa4b16b3:pedometer  Non-wakeup",\n              "value": [\n                {\n                  "intVal": 6248,\n                  "mapVal": []\n                }\n              ]\n            }\n          ]\n        }\n      ]\n    }\n  ]\n}\n'


In [77]:
first_session = r.json()
first_session

{'bucket': [{'startTimeMillis': '1618241849757',
   'endTimeMillis': '1618245502437',
   'dataset': [{'dataSourceId': 'derived:com.google.step_count.delta:com.google.android.gms:aggregated',
     'point': [{'startTimeNanos': '1618241849757000000',
       'endTimeNanos': '1618245502437000000',
       'dataTypeName': 'com.google.step_count.delta',
       'originDataSourceId': 'raw:com.google.step_count.cumulative:OnePlus:GM1915:33e928a4fa4b16b3:pedometer  Non-wakeup',
       'value': [{'intVal': 6248, 'mapVal': []}]}]}]}]}